In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import pandas as pd
data = pd.read_csv('/gdrive/MyDrive/big_project/data/data_preprocessing.csv', encoding='cp949')

# 빈도 컬럼 추가
freq = data['Destination (3)'].value_counts()
data['desfreq'] = data['Destination (3)'].map(freq)
data.head()

# 인덱스 생성
user2idx = {}
for i, l in enumerate(data['num'].unique()):
    user2idx[l] = i

item2idx = {}
for i, l in enumerate(data['Destination (3)'].unique()):
    item2idx[l] = i

idx2user = {i:user for user, i in user2idx.items()}
idx2item = {i:item for item, i in item2idx.items()}

# 인덱스 컬럼 생성
useridx = data['user_id'] = data['num'].apply(lambda x: user2idx[x]).values
itemidx = data['des_id'] = data['Destination (3)'].apply(lambda x: item2idx[x]).values

data

,num,category,T,car_type,is_freezer,vibration-free,transport_type,Departure (1),Departure (2),Departure (3),Destination (1),Destination (2),Destination (3),Destination type,desfreq,user_id,des_id
0,46640,3,3,1,0,2,3,11.0,166.0,송악읍,6.0,66.0,안영동,7.0,4,0,0
1,46669,3,3,1,0,2,2,8.0,111.0,팔탄면,8.0,111.0,동탄면,7.0,5,1,1
2,46673,3,6,1,0,2,3,9.0,119.0,동면,2.0,40.0,모라동,7.0,4,2,2
3,46676,5,3,1,0,2,3,6.0,67.0,관저동,4.0,50.0,북성동,6.0,2,3,3
4,46683,1,3,4,1,2,3,10.0,147.0,광혜원면,8.0,109.0,석정동,6.0,3,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4383,105113,3,6,1,0,2,3,3.0,45.0,봉덕동,3.0,45.0,대명동,7.0,1,4383,1878
4384,105249,5,3,1,0,2,2,11.0,151.0,영성,1.0,24.0,가락,6.0,69,4384,214
4385,105372,3,6,1,0,2,3,4.0,51.0,송림,4.0,51.0,송현,7.0,3,4385,833
4386,105534,5,6,1,0,2,3,3.0,45.0,봉덕동,3.0,48.0,호산동,6.0,3,4386,949


In [3]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=804889 sha256=d17c3ff2f03d4becd06b5c8d3e3bac497c58e033dcc0447fa0a0a9c41232eecf
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [4]:
from lightfm.data import Dataset
import numpy as np

# interaction
ratings_source = [(data['user_id'][i],
                   data['des_id'][i],
                   data['desfreq'][i]) for i in range(data.shape[0])]

# item_feautres
item_meta = data[['des_id', 'category', 'T', 'Departure (1)', 'Departure (2)', 'Departure (3)', 'Destination (1)', 'Destination (2)', 'Destination (3)']]
item_features_source = [(item_meta['des_id'][i],
                        np.array(item_meta.iloc[i,1:]).reshape(-1))
                        for i in range(item_meta.shape[0])]


# item_feature_source = [(item_meta['des_id'][i],
#                        {'category':item_meta['category'][i],
#                         'T':item_meta['T'][i],
#                         'Departure (1)':item_meta['Departure (1)'][i],
#                         'Departure (2)':item_meta['Departure (2)'][i],
#                         # 'Departure (3)':item_meta['Departure (3)'][i],
#                         'Destination (1)':item_meta['Destination (1)'][i],
#                         'Destination (2)':item_meta['Destination (2)'][i]
#                         # 'Destination (3)':item_meta['Destination (3)'][i]
#                         }) for i in range(item_meta.shape[0])]

# fit Dataset
dataset = Dataset()
dataset.fit(users=data['user_id'],
            items=data['des_id'])
interactions, weights = dataset.build_interactions(ratings_source)


dataset.fit(users=data['user_id'],
            items=data['des_id'],
            item_features=item_meta[item_meta.columns[1:]].values.flatten())
item_features=dataset.build_item_features(item_features_source)




In [ ]:
print(item_features)

  (0, 0)	0.030303031
  (0, 2)	0.030303031
  (0, 3)	0.18181819
  (0, 4)	0.030303031
  (0, 5)	0.030303031
  (0, 6)	0.121212125
  (0, 9)	0.030303031
  (0, 10)	0.030303031
  (0, 11)	0.030303031
  (0, 57)	0.030303031
  (0, 66)	0.121212125
  (0, 119)	0.030303031
  (0, 149)	0.030303031
  (0, 166)	0.030303031
  (0, 1879)	0.030303031
  (0, 1880)	0.121212125
  (0, 2068)	0.030303031
  (0, 2146)	0.030303031
  (0, 3236)	0.030303031
  (1, 1)	0.024390243
  (1, 2)	0.024390243
  (1, 3)	0.12195122
  (1, 5)	0.024390243
  (1, 6)	0.07317073
  (1, 7)	0.024390243
  :	:
  (1875, 1875)	0.11111111
  (1875, 3122)	0.11111111
  (1875, 4851)	0.11111111
  (1876, 1)	0.11111111
  (1876, 3)	0.11111111
  (1876, 8)	0.11111111
  (1876, 13)	0.11111111
  (1876, 78)	0.11111111
  (1876, 200)	0.11111111
  (1876, 1876)	0.11111111
  (1876, 2645)	0.11111111
  (1876, 4853)	0.11111111
  (1877, 3)	0.22222222
  (1877, 12)	0.22222222
  (1877, 169)	0.11111111
  (1877, 170)	0.11111111
  (1877, 1877)	0.11111111
  (1877, 2565)	0.11111111


In [9]:
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k
from lightfm import LightFM
# Split Train, Test data
model = LightFM(no_components=150,
                    learning_schedule='adadelta',
                    loss='warp-kos',
                    learning_rate=0.05,
                    random_state=2024)
(train, test) = random_train_test_split(interactions=interactions, test_percentage=0.2)
model.fit(train, item_features=item_features, epochs=50)

train_precision = precision_at_k(model, train, k=5, item_features=item_features).mean()
test_precision = precision_at_k(model, test, k=5, item_features=item_features).mean()
train_auc = auc_score(model, train, item_features=item_features).mean()
test_auc = auc_score(model, test, item_features=item_features).mean()
# test_auc = auc_score(model, test, item_features=item_features).mean()
output = -test_precision #precision은 높을수록 좋음 : 높은 숫자를 찾기위해 음수화

print( 'Precision: train %.4f, test %.4f, AUC: train %.4f, test %.4f.'
      % ( train_precision,test_precision,train_auc, test_auc))


Precision: train 0.1690, test 0.0139, AUC: train 0.9988, test 0.6144.


In [10]:
train_precision = precision_at_k(model, train, k=5, item_features=item_features)

In [ ]:
print(train_precision.mean())

0.16752139


In [ ]:
score = auc_score(
        model, interactions,
        item_features=item_features,
        num_threads=4).mean()

In [ ]:
print(score)

0.9210499


In [ ]:
scores = model.predict(0,data['des_id'].unique(), item_features=item_features)

temp = {}
temp['city'] = [idx2item[i] for i in range(len(scores))]
temp['scores'] = [l for l in scores]
temp = pd.DataFrame(temp)
temp_data = temp.sort_values(by='scores', ascending=False)[:30]
temp_data
# print(f'num : 46640 Recommend : ')
# display(temp_data)

,city,scores
5,가락동,2.898542
821,옥포,2.580406
1562,성연,2.508346
1407,지곡,2.504447
793,대신,2.432241
550,능동,2.429352
12,중앙동,2.401737
923,용계,2.380035
43,초월읍,2.374412
780,송악읍,2.367352


In [7]:
from lightfm import LightFM
from hyperopt import fmin, hp, tpe, Trials
from lightfm.evaluation import precision_at_k
import numpy as np
from lightfm.cross_validation import random_train_test_split

(train, test) = random_train_test_split(interactions=interactions, test_percentage=0.2)

# Define Search Space
trials = Trials()
space = [hp.choice('no_components', range(10, 50, 10)),
         hp.uniform('learning_rate', 0.01, 0.05)]

# Define Objective Function
def objective(params):
    no_components, learning_rate = params

    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(interactions=train,
              item_features=item_features,
              epochs=30,
              verbose=False)

    test_precision = precision_at_k(model, test, k=5, item_features=item_features).mean()
    print("no_comp: {}, lrn_rate: {:.5f}, precision: {:.5f}".format(
      no_components, learning_rate, test_precision))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)


no_comp: 40, lrn_rate: 0.04885, precision: 0.01777
no_comp: 40, lrn_rate: 0.01904, precision: 0.01777
no_comp: 10, lrn_rate: 0.03628, precision: 0.01800
no_comp: 10, lrn_rate: 0.03235, precision: 0.01777
no_comp: 40, lrn_rate: 0.03050, precision: 0.01777
no_comp: 10, lrn_rate: 0.01828, precision: 0.01777
no_comp: 40, lrn_rate: 0.01853, precision: 0.01777
no_comp: 20, lrn_rate: 0.01777, precision: 0.01777
no_comp: 30, lrn_rate: 0.02261, precision: 0.01777
no_comp: 10, lrn_rate: 0.01535, precision: 0.01777
100%|██████████| 10/10 [00:13<00:00,  1.35s/trial, best loss: -0.017995445057749748]


In [8]:
best_params

{'learning_rate': 0.03627562237749584, 'no_components': 0}

In [ ]:
dataset.model_dimensions()

(4388, 1887)